# 7주차 모의고사

#### 데이터
- 1번 문제 : TI_train / TI_test / TI_submission
- 2번 문제 : tc_train / tc_test / tc_submission
- 3번 문제 : Train / Test / sample_submission
***
# 1번 문제

## 1-1. AnnualIncome을 표준화 및 정규화 한 후 표준화 결과의 1사분위수와 정규화 결과의 3사분위수의 합을 구하시오.

In [1]:
import pandas as pd
import numpy as np

In [75]:
train = pd.read_csv('TI_train.csv')
test = pd.read_csv("TI_test.csv")
submission = pd.read_csv("TI_submission.csv")

In [76]:
train.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
3,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0
4,25,Private Sector/Self Employed,No,1150000,4,0,No,No,0


In [77]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [78]:
m_s = MinMaxScaler()
s_s = StandardScaler()

In [79]:
train['m_ac'] = m_s.fit_transform(train[['AnnualIncome']]) # 정규화
train['s_ac'] = s_s.fit_transform(train[['AnnualIncome']]) # 표준화

In [80]:
q1 = np.percentile(train['s_ac'], 25)
q3 = np.percentile(train['m_ac'], 75)

In [81]:
q1, q3

(-0.9020329067914399, 0.6333333333333333)

In [82]:
v = q1 + q3

In [83]:
v

-0.2686995734581066

## 1-2. 더 많은 비중을 차지하는 Employment Type의 Age의 표준편차와 30대의 AnnualIncome 평균을 곱한 값을 출력하시오.

In [84]:
# train.drop(['m_ac', 's_ac'], axis = 1, inplace = True)
train = train.iloc[:, :-2]

In [85]:
t = train['Employment Type'].value_counts().index[0]

In [86]:
v1 = train[train['Employment Type'] == t].Age.std()

In [87]:
v1

2.978209360744764

In [88]:
age_df = train[train.Age.isin(np.arange(30, 40))]

In [89]:
age_df.Age.min(), age_df.Age.max()

(30, 35)

In [90]:
v2 = age_df['AnnualIncome'].mean()

In [91]:
f = v1 * v2

In [92]:
f

2834746.8366601076

## 1-3. TravelInsuarance를 예측하는 모델링을 수행하시오. submission파일에 확률값으로 제출하고 "result1"이라는 파일명으로 저장하시오.

In [93]:
train.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
3,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0
4,25,Private Sector/Self Employed,No,1150000,4,0,No,No,0


In [95]:
train['Employment Type'] = [1 if t == 'Government Sector' else 0 for t in train['Employment Type']]
test['Employment Type'] = [1 if t == 'Government Sector' else 0 for t in test['Employment Type']]

In [97]:
train['GraduateOrNot'] = [1 if g == 'Yes' else 0 for g in train['GraduateOrNot']]
test['GraduateOrNot'] = [1 if g == 'Yes' else 0 for g in test['GraduateOrNot']]

In [98]:
train['FrequentFlyer'] = [1 if g == 'Yes' else 0 for g in train['FrequentFlyer']]
test['FrequentFlyer'] = [1 if g == 'Yes' else 0 for g in test['FrequentFlyer']]

In [99]:
train['EverTravelledAbroad'] = [1 if g == 'Yes' else 0 for g in train['EverTravelledAbroad']]
test['EverTravelledAbroad'] = [1 if g == 'Yes' else 0 for g in test['EverTravelledAbroad']]

In [39]:
#train = pd.get_dummies(train)
#test = pd.get_dummies(test)

In [40]:
train.shape, test.shape

((1500, 13), (487, 13))

In [41]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [102]:
X = train.iloc[:, :-1]
y = train.TravelInsurance

target = test[X.columns]

In [104]:
scaled_X = m_s.fit_transform(X)
scaled_target = m_s.transform(target)

In [108]:
X_train, X_val, y_train, y_val = train_test_split(scaled_X, y, test_size = 0.2, stratify = y, random_state = 42)

In [107]:
rf = RandomForestClassifier(random_state = 42)

In [109]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [110]:
scaled_pred = rf.predict_proba(X_val)[:, 1]

In [111]:
roc = roc_auc_score(y_val, scaled_pred)

In [112]:
roc

0.7936439043209877

In [43]:
X.head()

,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No
0,31,400000,6,1,0,1,0,0,1,1,0,1
1,31,1250000,7,0,0,0,1,0,1,1,0,1
2,28,700000,3,1,0,0,1,0,1,1,0,1
3,28,700000,8,1,0,0,1,0,1,0,1,1
4,25,1150000,4,0,0,0,1,1,0,1,0,1


In [44]:
target.head()

,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No
0,28,750000,5,1,0,0,1,0,1,1,0,1
1,26,1400000,4,1,1,0,1,0,1,1,0,0
2,29,1200000,7,0,0,0,1,0,1,1,0,0
3,33,400000,4,1,0,1,0,0,1,1,0,1
4,34,1500000,3,0,1,0,1,0,1,0,1,0


### Hold Out

In [45]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

In [46]:
rf = RandomForestClassifier(random_state = 42, max_depth = 4, n_estimators = 200)

In [47]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, n_estimators=200, random_state=42)

In [48]:
rf_pred = rf.predict_proba(X_val)[:, 1]

In [49]:
rf_roc = roc_auc_score(y_val, rf_pred)

In [50]:
rf_roc

1.0

In [58]:
sub_rf = rf.predict_proba(target)[:, 1]

In [51]:
lgbm = LGBMClassifier(random_state = 42, max_depth = 5, n_estimators = 200, learning_rate = 0.1)

In [52]:
lgbm.fit(X_train, y_train)

LGBMClassifier(max_depth=5, n_estimators=200, random_state=42)

In [53]:
lgbm_pred = lgbm.predict_proba(X_val)[:, 1]

In [54]:
lgbm_roc = roc_auc_score(y_val, lgbm_pred)

In [55]:
lgbm_roc

1.0

In [59]:
sub_lgbm = lgbm.predict_proba(target)[:, 1]

In [72]:
submission['TravelInsurance'] = (sub_rf + sub_lgbm) / 2

In [73]:
submission.head()

,index,TravelInsurance
0,0,0.038109
1,1,0.976056
2,2,0.075057
3,3,0.032320
4,4,0.987567


In [63]:
#submission.to_csv('result1.csv', index = False)

###  Ensemble

In [64]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [65]:
rf_pred = np.zeros(target.shape[0])
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = RandomForestClassifier(random_state = 42)
    rf.fit(tr_x, tr_y)
    
    val_pred = rf.predict_proba(val_x)[:, 1]
    val_score = roc_auc_score(val_y, val_pred)
    print(val_score)
    
    fold_pred = rf.predict_proba(target)[:, 1] / 20
    rf_pred += fold_pred

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [66]:
lgbm_pred = np.zeros(target.shape[0])
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42)
    lgbm.fit(tr_x, tr_y)
    
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    val_score = roc_auc_score(val_y, val_pred)
    print(val_score)
    
    fold_pred = lgbm.predict_proba(target)[:, 1] / 20
    lgbm_pred += fold_pred

1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0


In [68]:
rf_pred[:5]

array([0.    , 0.4995, 0.0135, 0.0035, 0.5   ])

In [69]:
lgbm_pred[:5]

array([7.95829726e-06, 4.99986201e-01, 7.95829726e-06, 7.95829726e-06,
       4.99986201e-01])

In [70]:
submission['TravelInsurance'] = rf_pred + lgbm_pred

In [71]:
submission.head()

,index,TravelInsurance
0,0,0.000008
1,1,0.999486
2,2,0.013508
3,3,0.003508
4,4,0.999986


In [74]:
#submission.to_csv('result1.csv', index = False)

***
# 2번 문제

## 2-1. AccountWeeks에 대해 이상치를 탐색하고 이상치 데이터의 DayMins의 중위값을 출력하시오.

In [113]:
train = pd.read_csv('tc_train.csv')
test = pd.read_csv('tc_test.csv')
submission = pd.read_csv('tc_submission.csv')

In [114]:
train.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [115]:
test.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,113,1,0,0.00,1,155.0,93,55.0,16.53,13.5
1,0,67,1,0,0.00,0,109.1,117,38.0,10.87,12.8
2,1,98,1,0,0.00,4,0.0,0,14.0,7.98,6.8
3,0,147,1,0,0.33,1,212.8,79,57.3,10.21,10.2
4,0,96,1,0,0.30,1,144.0,102,47.0,11.24,10.0


In [116]:
submission.head()

,index,Churn
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


### 이상치 탐색 문제
- 1. q1, q3를 구한다.
- 2. iqr(qe - q1)을 구한다.
- 3. 상한선, 하한선을 구한다.
- 4. 문제에서 요구하는 접근법을 실행한다.

In [118]:
q1 = np.percentile(train.AccountWeeks, 25)
q3 = np.percentile(train.AccountWeeks, 75)

In [119]:
iqr = q3 - q1

In [120]:
l_b = q1 - 1.5 * iqr
u_b = q3 + 1.5 * iqr

In [122]:
out = train[(train.AccountWeeks < l_b) | (train.AccountWeeks > u_b)]

In [125]:
m = out.DayMins.median()

In [126]:
m

171.5

## 2-2. MonthlyCharge에 대해 표준화를 하고 결과가 0.7초과, 0.3미만인 데이터의 OverageFee의 3분위수를 출력하시오.

In [128]:
train.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [129]:
ss = StandardScaler()

In [130]:
train['ss_mc'] = ss.fit_transform(train[['MonthlyCharge']])

In [133]:
result = train[(train.ss_mc > 0.7) | (train.ss_mc < 0.3)]

In [134]:
q3 = np.percentile(result.OverageFee, 75)

In [135]:
q3

11.662500000000001

## 2-3. Churn을 예측하는 모델링을 수행하시오. submission형태로 제출하며 test 데이터의 churn 값은 0 혹은 1로 변환하여 'result2'라는 파일명으로 저장하시오.

In [137]:
train.drop('ss_mc', axis = 1, inplace = True)

In [139]:
ms = MinMaxScaler()

In [147]:
X = train.drop('Churn', axis = 1)
y = train.Churn

In [148]:
target = test[X.columns]

In [149]:
scaled_x = ms.fit_transform(X)
scaled_target = ms.transform(target)

In [152]:
X = pd.DataFrame(scaled_x, columns = X.columns)
target = pd.DataFrame(scaled_target, columns = target.columns)

In [153]:
X.head()

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0.524793,1.0,1.0,0.500000,0.111111,0.755701,0.666667,0.766736,0.542606,0.500
1,0.438017,1.0,1.0,0.685185,0.111111,0.460661,0.745455,0.693515,0.537658,0.685
2,0.561983,1.0,0.0,0.000000,0.000000,0.693843,0.690909,0.379707,0.333150,0.610
3,0.342975,0.0,0.0,0.000000,0.222222,0.853478,0.430303,0.432008,0.170423,0.330
4,0.305785,0.0,0.0,0.000000,0.333333,0.475200,0.684848,0.264644,0.407916,0.505


In [154]:
target.head()

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0.462810,1.0,0.0,0.000000,0.111111,0.441847,0.563636,0.411088,0.908741,0.675
1,0.272727,1.0,0.0,0.000000,0.000000,0.311003,0.709091,0.233264,0.597581,0.640
2,0.400826,1.0,0.0,0.000000,0.444444,0.000000,0.000000,-0.017782,0.438703,0.340
3,0.603306,1.0,0.0,0.061111,0.111111,0.606613,0.478788,0.435146,0.561297,0.510
4,0.392562,1.0,0.0,0.055556,0.111111,0.410490,0.618182,0.327406,0.617922,0.500


In [163]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [162]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [199]:
%%time
LR_acc = []
LR_pred = np.zeros(target.shape[0])
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    ### Fitting
    LR = LogisticRegression()
    LR.fit(tr_x, tr_y)
    
    ### Validation
    val_pred = LR.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.45 else 0 for p in val_pred]
    val_acc = accuracy_score(val_y, val_pred)
    LR_acc.append(val_acc)
    
    ### Prediction
    fold_pred = LR.predict_proba(target)[:, 1] / 20
    LR_pred + fold_pred
print(np.mean(LR_acc))

0.8603333333333334
CPU times: user 264 ms, sys: 574 ms, total: 838 ms
Wall time: 186 ms


In [198]:
%%time
svm_acc = []
svm_pred = np.zeros(target.shape[0])
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    ### Fitting
    svm = SVC(probability = True)
    svm.fit(tr_x, tr_y)
    
    ### Validation
    val_pred = svm.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.45 else 0 for p in val_pred]
    val_acc = accuracy_score(val_y, val_pred)
    svm_acc.append(val_acc)
    
    ### Prediction
    fold_pred = svm.predict_proba(target)[:, 1] / 20
    svm_pred + fold_pred
print(np.mean(svm_acc))

0.9119999999999999
CPU times: user 2.18 s, sys: 8.48 ms, total: 2.19 s
Wall time: 2.19 s


In [193]:
final_pred = LR_pred + svm_pred

In [194]:
final_pred = [1 if p >= 0.45 else 0 for p in final_pred]

In [195]:
submission['Churn'] = final_pred

In [196]:
submission.head()

,index,Churn
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [197]:
round(accuracy_score(test['Churn'], final_pred) * 100, 3)

83.784

In [178]:
#submission.to_csv("result2.csv", index = False)

***
# 3번 문제

## 3-1. POSTED_BY 값에 따른 SQUARE_FT의 표준편차의 최솟값을 구하시오.

In [200]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
submission = pd.read_csv("sample_submission.csv")

In [201]:
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [202]:
test.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE
0,Owner,0,0,1,BHK,545.171340,1,1,"Kamrej,Surat",21.262000,73.047700
1,Dealer,1,1,2,BHK,800.000000,0,0,"Panvel,Lalitpur",18.966114,73.148278
2,Dealer,0,0,2,BHK,1257.096513,1,1,"New Town,Kolkata",22.592200,88.484911
3,Dealer,0,0,3,BHK,1400.329489,1,1,"Kalwar Road,Jaipur",26.988300,75.584600
4,Owner,0,0,1,BHK,430.477830,1,1,"Mai Mandir,Nadiad",22.700000,72.870000


In [205]:
train.groupby('POSTED_BY').SQUARE_FT.std().min()

20783.648773831006

## 3-2. SQUARE_FT에 대해 정규화를 한 후 내림차순을 기준으로 상위 21~30위의 값의 TARGET 평균을 구하시오.

In [206]:
ms = MinMaxScaler()

In [207]:
train['ms_sf'] = ms.fit_transform(train[['SQUARE_FT']])

In [213]:
sorted_result = train.sort_values(by = 'ms_sf', ascending = False)

In [214]:
sorted_result.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),ms_sf
10745,Owner,0,0,3,BHK,2.545455e+08,1,1,"Malur,Bangalore",13.021000,77.938000,28000.0,1.000000
11238,Dealer,0,0,3,BHK,1.875000e+08,1,1,"R.T. Nagar,Bangalore",13.018900,77.596300,30000.0,0.736607
15724,Owner,0,0,2,BHK,8.064516e+07,1,1,"Lakkasandra,Bangalore",12.795926,77.331535,25000.0,0.316820
25241,Owner,0,0,3,BHK,4.300000e+06,1,1,"New Kumaran Nagar,Chennai",10.810426,78.668914,43.0,0.016893
28591,Owner,0,0,2,BHK,8.750000e+05,1,1,"Ajit Singh Nagar,Vijayawada",30.466084,74.501902,35.0,0.003437


In [219]:
sorted_result.iloc[20:30, -2].mean() # 21위 ~ 30위의 평균

9148.0

In [224]:
sorted_result.iloc[20:30].index # 21위 ~ 30위

Int64Index([13955, 22687, 27036, 25264, 15225, 13661, 28049, 29002, 4866,
            27998],
           dtype='int64')

In [ ]:
sorted_result.iloc[19:30].index # 20위 ~ 30위

## 3-3. TARGET을 예측하는 모델링을 수행하시오. submission 파일명은 'result3'으로 지정하시오.

In [226]:
train = train.iloc[:, :-1]

In [227]:
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [228]:
train.drop('ADDRESS', axis = 1, inplace = True)
test.drop('ADDRESS', axis = 1, inplace = True)

In [230]:
train.POSTED_BY.unique()

array(['Owner', 'Dealer', 'Builder'], dtype=object)

In [231]:
train.BHK_OR_RK.unique()

array(['BHK', 'RK'], dtype=object)

In [232]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [233]:
train.shape, test.shape

((29451, 14), (68720, 13))

In [238]:
X = train.drop('TARGET(PRICE_IN_LACS)', axis = 1)
y = train['TARGET(PRICE_IN_LACS)']

In [239]:
target = test[X.columns]

In [242]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [243]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [248]:
LR_pred = np.zeros(target.shape[0])
LR_rmse = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    LR = LinearRegression()
    LR.fit(tr_x, tr_y)
    
    val_pred = LR.predict(val_x)
    val_rmse = mean_squared_error(val_y, val_pred) ** 0.5
    LR_rmse.append(val_rmse)
    fold_pred = LR.predict(target) / 5
    LR_pred += fold_pred
print(np.mean(LR_rmse))

588.3565805277019


In [249]:
xgb_pred = np.zeros(target.shape[0])
xgb_rmse = []
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb = XGBRegressor(random_state = 42)
    xgb.fit(tr_x, tr_y)
    
    val_pred = xgb.predict(val_x)
    val_rmse = mean_squared_error(val_y, val_pred) ** 0.5
    xgb_rmse.append(val_rmse)
    fold_pred = xgb.predict(target) / 5
    xgb_pred += fold_pred
print(np.mean(xgb_rmse))

176.19256361174683


In [251]:
submission['TARGET(PRICE_IN_LACS)'] = (LR_pred * 0.2 + xgb_pred * 0.8) / 2

In [253]:
submission['TARGET(PRICE_IN_LACS)'] = [abs(x) if x < 0 else x for x in submission['TARGET(PRICE_IN_LACS)']]

In [254]:
submission

,TARGET(PRICE_IN_LACS)
0,0.095829
1,101.751279
2,31.771084
3,34.945169
4,2.360265
...,...
68715,60.213995
68716,60.381267
68717,3589.850568
68718,44.550468


In [255]:
#submission.to_csv('result3.csv', index = False)